In [52]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [53]:
api_wrapper_wiki= WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

wiki.name

'wikipedia'

In [54]:
api_wrapper_arxiv= ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [55]:
tools=[wiki, arxiv]

### Rag tool

In [56]:
# Custom tools[Rag]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [57]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [58]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = groq_api_key


In [59]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [60]:
loader = WebBaseLoader("https://imdeepak.in/")
docs= loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0).split_documents(docs)
vectordb=FAISS.from_documents(documents, embeddings)
retriver= vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7feb5c061e70>, search_kwargs={})

In [61]:
#create retriver tool
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, "imdeepak-tool","personal website")
retriver_tool.name


'imdeepak-tool'

In [62]:
tools=[wiki, arxiv, retriver_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/var/www/learning/genAIApps/langchain-framework/langenv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='imdeepak-tool', description='personal website', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7fec75525c60>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7feb5c061e70>, sea

In [64]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")

In [67]:
#prompt using hub
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [76]:
#Agents
from langchain.agents import create_openai_tools_agent
agents = create_openai_tools_agent(llm=llm,  tools=tools, prompt=prompt)  
agents

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [81]:
from langchain.agents import AgentExecutor
agent_execute = AgentExecutor(agent=agents, tools=tools, verbose=True)
agent_execute

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [82]:
agent_execute.invoke({"input": "who is deepak ?"})



> Entering new AgentExecutor chain...

Invoking: `imdeepak-tool` with `{'query': 'deepak'}`


Deepak Kumar | Expert React & JavaScript Developer✕Connect with Me!+91-8285257636dpk1391981@gmail.comDeepak KumarMy Professional Software Saga/ | Me | Experience | Education | Skills | Deepak KumarJavaScript | Full Stack | Node js | React js | Angular Js | MySql | NoSqlExperienced Software Engineer with  7+ years  of experience, combining technical expertise, strategic problem-solving, and leadership abilities. Skilled in guiding teams to deliver scalable solutions, optimizing processes, and adapting to new technologies. Passionate about creating innovative solutions and driving success in the fast-paced software development landscape.Contact me Technical ExperienceInstant System IncSr Software Engineer ( 7+ years )2017 - currentInstant Systems Inc is a unique software product incubator that helps ambitious start-ups with building of innovative products and passionate teams around the world,

{'input': 'who is deepak ?',
 'output': 'Deepak Kumar is a software engineer with 7+ years of experience.  \n'}

In [85]:
agent_execute.invoke({"input": "what is the latest research on transformers ?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'latest research on transformers'}`


Published: 2024-06-03
Title: Reproducibility Study on Adversarial Attacks Against Robust Transformer Trackers
Authors: Fatemeh Nourilenjan Nokabadi, Jean-François Lalonde, Christian Gagné
Summary: New transformer networks have been integrated into obI'm sorry, I don't have access to the full content of research papers. However, the provided information suggests that recent research on transformers focuses on improving their robustness against adversarial attacks.  Would you like me to search for more information about transformer trackers or adversarial attacks? 


> Finished chain.


{'input': 'what is the latest research on transformers ?',
 'output': "I'm sorry, I don't have access to the full content of research papers. However, the provided information suggests that recent research on transformers focuses on improving their robustness against adversarial attacks.  Would you like me to search for more information about transformer trackers or adversarial attacks? \n"}

In [86]:
agent_execute.invoke({"input": "what is paper 1706.03762 about ?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThis paper, "Attention Is All You Need",  introduces a novel network architecture called the Transformer, which relies entirely on attention mechanisms to compute representations of input sequences. This architecture has achieved state-of-the-art results on various machine translation tasks and shows promise for other sequence transduction tasks as well. 


> Finished chain.


{'input': 'what is paper 1706.03762 about ?',
 'output': 'This paper, "Attention Is All You Need",  introduces a novel network architecture called the Transformer, which relies entirely on attention mechanisms to compute representations of input sequences. This architecture has achieved state-of-the-art results on various machine translation tasks and shows promise for other sequence transduction tasks as well. \n'}